In [1]:
import os
import sys
import datetime

In [2]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../eventpredictor'))

from eventpredictor.config import RAW_DATA_DIR
from eventpredictor.data.download_data import FootballScraper

2024-05-29 14:04:13.088 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor


2024-05-29 14:04:13.096 | INFO     | eventpredictor.config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor


In [3]:
fs = FootballScraper(
    start_date=datetime.date(2015, 5, 31),
    end_date=datetime.date(2024, 1, 31),
    raw_data_dir=RAW_DATA_DIR
)

In [4]:
fs.download_events()
fs.convert_events()
fs.combine_events()

In [5]:
fs.download_odds()
fs.conert_odds()
fs.combine_odds()

In [9]:
import pandas as pd

In [57]:
cols_to_select = [
    'id',
    'tournament_slug',
    'tournament_category_slug',
    'status_description',
    'status_type',
]

events_df = pd.read_csv(os.path.join(fs.scheduled_events, 'scheduled_events.csv'), usecols=cols_to_select)

In [58]:
events_df = events_df.drop_duplicates()
events_df.shape

(473192, 5)

In [59]:
events_df.head()

,id,tournament_slug,tournament_category_slug,status_description,status_type
0,7811264,afc-champions-league-knockout-stage,asia,AP,finished
1,7979684,serie-c-girone-c,italy,Ended,finished
2,7979686,serie-c-girone-c,italy,Canceled,canceled
3,7979685,serie-c-girone-c,italy,Ended,finished
4,7979681,serie-c-girone-c,italy,Canceled,canceled


In [60]:
events_df['tournament_category_slug'].value_counts()

tournament_category_slug
world                 32841
england               27990
italy                 20412
turkey                20297
germany-amateur       18648
                      ...  
dominican-republic        1
belgium-amateur           1
barbados                  1
international             1
french-guiana             1
Name: count, Length: 177, dtype: int64

In [61]:
countries_to_remove = {
    'africa',
    'asia',
    'esoccer',
    'europe',
    'international',
    'north-central-america',
    'oceania',
    'simulated-reality-women',
    'south-africa',
    'south-america',
    'world'
}

In [62]:
events_df = events_df[~events_df['tournament_category_slug'].isin(countries_to_remove)]

In [63]:
events_df['tournament_category_slug'].value_counts()

tournament_category_slug
england               27990
italy                 20412
turkey                20297
germany-amateur       18648
brazil                13246
                      ...  
djibouti                  1
dominican-republic        1
belgium-amateur           1
barbados                  1
french-guiana             1
Name: count, Length: 166, dtype: int64

In [64]:
selected_tournaments = events_df[['tournament_category_slug', 'tournament_slug', 'id']].groupby(['tournament_category_slug', 'tournament_slug'], as_index=False)['id'].count()
selected_tournaments = selected_tournaments[selected_tournaments['id'] > 2000].reset_index(drop=True).rename(columns={'id': 'count'})
selected_tournaments.head()

,tournament_category_slug,tournament_slug,count
0,algeria,ligue-1,2315
1,argentina,liga-profesional,3189
2,argentina,primera-b-nacional,2397
3,belarus,vysshaya-league,2091
4,belgium,pro-league,2363


In [65]:
selected_tournaments_set = (selected_tournaments['tournament_category_slug'] + selected_tournaments['tournament_slug']).values

In [66]:
events_df = events_df.loc[(events_df['tournament_category_slug'] + events_df['tournament_slug']).isin(selected_tournaments_set)]
events_df = events_df.reset_index(drop=True)
events_df.head()

,id,tournament_slug,tournament_category_slug,status_description,status_type
0,7979684,serie-c-girone-c,italy,Ended,finished
1,7979686,serie-c-girone-c,italy,Canceled,canceled
2,7979685,serie-c-girone-c,italy,Ended,finished
3,7979681,serie-c-girone-c,italy,Canceled,canceled
4,8177688,brasileirao-serie-b,brazil,Ended,finished


In [67]:
events_df.shape

(181373, 5)

In [68]:
events_df['status_description'].value_counts()

status_description
Ended                172796
Canceled               6202
Postponed              2331
Abandoned                15
AP                       10
Retired                   7
Walkover                  7
AET                       4
Coverage canceled         1
Name: count, dtype: int64

In [69]:
events_df = events_df[events_df['status_description'].eq('Ended')]
events_df = events_df.reset_index(drop=True)
events_df.head()

,id,tournament_slug,tournament_category_slug,status_description,status_type
0,7979684,serie-c-girone-c,italy,Ended,finished
1,7979685,serie-c-girone-c,italy,Ended,finished
2,8177688,brasileirao-serie-b,brazil,Ended,finished
3,7834147,mozzart-bet-superliga,serbia,Ended,finished
4,7834145,mozzart-bet-superliga,serbia,Ended,finished


In [70]:
events_df.shape

(172796, 5)

In [73]:
temp_df = events_df[['id']].sort_values('id').reset_index(drop=True)
temp_df.head()

,id
0,5617262
1,5617268
2,5646264
3,5646266
4,5646270


In [75]:
temp_df.to_csv(os.path.join(RAW_DATA_DIR, 'download.csv'), index=False)